# Tarea #3

## El principito: regex y los $n$-grams

¿listo para tu tercera tarea en Python? En esta ocasión analizaremos la historia del *Principito* por Antoine de Saint-Exupéry usando las herramientas de Python que hemos visto hasta el momento usando herramientas básicas de NLP.

**NLP** o *Natural Language Processing* , es una rama de la inteligencia artificial y ciencia de la computación que se encarga de procesar y cuantificar grandes cantidades de palabras.

In [2]:
%%bash
head -10 files/principito.txt

El Principito

Antoine De Saint Exupery

Capítulo 1
Cuando yo tenía seis años vi en un libro sobre la selva virgen que se
titulaba "Historias vividas", una magnífica lámina. Representaba una
serpiente boa que se tragaba a una fiera. Esta es la copia del dibujo.
En el libro se afirmaba: "La serpiente boa se traga su presa entera,
sin masticarla. Luego ya no puede moverse y duerme durante los


In [ ]:
# Corre esta linea antes de empezar la tarea.
# **** Es importante instalar desde 'pip' el paquete 
# unidecode si aún no lo tienes. Para esto, corre en la línea de
# comandos:
#          pip install unidecode
from unidecode import unidecode
import re
from collections import Counter

def clean_book(file):
    with open(file, "r", encoding="utf-8") as f:
        txtfile = f.read()
    # Removemos acentos y ñ por n para hacer
    # las búsquedas más faciles
    return unidecode(txtfile).replace("\n", " ")

principito = clean_book("files/principito.txt")

¿Cuántas veces se menciona `principito` en la lectura?

¿Cuáles son los verbos en [pretérito imperfecto](https://es.wikipedia.org/wiki/Pretérito_imperfecto) que más se mencionan en el principito? e.g., `jugaba`, `Miraba`, `hablaba`.

¿Quiénes son los 10 personajes que más se mencionan? Asume que al mencionar un personaje se refiere a `el` o `la` seguido del personaje. e.g., `la flor`, `el farolero`.

¿Qué se dice sobre las flores? Encuentra los adjetivos de `flores son (...)`

Completa la frase: `La prueba de(...)queria un cordero`. ¿Qué va dentro de `(...)`?

¿Qué tanto `...-dijo el principito`? e.g., 
`'-Tengo sed de esta agua -dijo el principito'`, `'-Tienen mucha prisa -dijo el principito`

# El modelo $n$-gram

En esta segunda parte de la tarea aplicaremos nuestros conocimiento de Python para implementar un modelo de machine learning aplicado a *NLP*: [$n$-grams](https://en.wikipedia.org/wiki/N-gram).

La finalidad de un modelo $n$-gram es modelar una secuencia de $n$ elementos (palabras, en este caso) dada una muestra de texto. Para lograr esto, el modelo asume que la probabilidad del acontecimiento de una palabra, dada toda una historia de palabras que la preceden, es igual a la probabilidad de esa misma palabra dada las últimas $n$ palabras que se dijeron. Esto es,

Dado un texto (o corpus) con $m$ palabras y $n \leq m$,

$$
    \mathbb P(W_m \ | \ W_{m-1}, W_{m-2}, \ldots, W_{m-n+1}) = \mathbb P(W_m \ | \  W_{m-1}, W_{m-2}, \ldots, W_{1})
$$

Dado esto, vemos que un $1$-gram (o *unigram* asume que)
$$
\mathbb P(X_m) = \mathbb P(W_m \ | \  W_{m-1}, W_{m-2}, \ldots, W_{1})
$$

un $2$-gram (o bigram), el cuál cumple la [propiedad de Markov](https://es.wikipedia.org/wiki/Cadena_de_Márkov), asume que,
$$
\mathbb P(X_m \ | \ X_{m-1}) = \mathbb P(W_m \ | \  W_{m-1}, W_{m-2}, \ldots, W_{1})
$$

un $3$-gram,
$$
\mathbb P(X_m \ | \ X_{m-1}, X_{m-2}) = \mathbb P(W_m \ | \  W_{m-1}, W_{m-2}, \ldots, W_{1})
$$

y así sucesivamente.

Este modelo se usaba en algunas apps en para predecir texto a pesar de su (aparente) deficiente suposición. En esta tarea implementaremos nuestro propio modelo $n$-gram para poder hablar como el principito.

Para lograr esto, no consideraremos ningún signo de puntuación dentro del cuento del principito, por lo que el primer paso será limpiar los datos.

In [9]:
from string import punctuation
# El primer paso será eliminar cada signo de puntuación.
principito = re.sub("[{}]".format(punctuation), "", principito)
# Paso siguiente será poner cada caractér en minúscula
principito = principito.lower()

### Unigram

Calcula la probabilidad de cada una de las palabras:

1) Crea la variable `tokens` la cual es una lista ordenada de las palabras dentro de la variable `principito`. Por ejemplo, si `principito = "esta es la historia del principito"`, `tokens` deberá ser `['esta', 'es', 'la', 'historia', 'del', 'principito']`

2) crea la variable `gram1`: un diccionario cuya llave es una palabra dentro de tokens y su valor es la probabilidad de su ocurrencia. Hint: usa la función `Counter` para lograr esto.

Asegurate de haber calculado correctamente las probabilidades. ¿Cuál es la suma de cada una de las probabilidades?

**Nota:** de tener un número marginalmente por arriba del 1, e.g., `1.0000000000000335`, como resultado, esto se debe a un [Roundoff Error](http://mathworld.wolfram.com/RoundoffError.html).

¿Cuál es la probabilidad de que aparezca la palabra `principito` en el texto?

¿Cuál es la probabilidad de la palabra `amigo` o `amigos` en el texto?

¿Cuál es la probabilidad de la palabra `el principito` en el texto?

¿Cuál es la probabilidad de la palabra `una flor` en el texto

Escribe la función `gram1prob` que tome una oración y un diccionario de probabilidades unigram, e.g. `gram1`, para calcular la probabilidad de la oración. **Hint** considera la función [`all`](https://docs.python.org/3/library/functions.html#all) para resolver el problema.

```python
>>> gram1prob("juez", gram1)
7.401924500370096e-05

>>> gram1prob("el principito era", gram1)
1.8159962136265147e-06

>>> gram1prob("un gran animal", gram1)
0
```

In [4]:
def gram1prob(sentence, gram):
    pass

### Bigram

A diferencia del unigram, si $n \geq 2$, rompemos con la supoción de independencia en las palabras. En esta segunda parte crearemos una función $bi$-gram $n=2$, esto es, supondremos que la probabilidad de una palabra está dada por la última escrita antes de esta. Por ejemplo, dada la oración

**"who controls the past controls the future who controls the present controls the past"**, nuestros *bi*grams serían

> "who controls"

> "controls the"

> "the past"

> "past controls"

> ...

> "controls the"

> "the past"

Tu tarea será crear la función `gram2prob` que, dada una lista de tokens (lista ordenada de las palabras), nos regrese un diccionario con la probabilidad de cada segunda palabra dada la última. Cada llave será la palabra anterior y cada valor otro diccionario cuya llave es la palabra que sigue y su valor la probabilidad de ocurrencia.

Por ejemplo, para el conjunto de palabras anterior, `gram2prob` nos debería regresar lo siguiente:
```python
defaultdict(list,
            {'controls': {'the': 1.0},
             'future': {'who': 1.0},
             'past': {'controls': 1.0},
             'present': {'controls': 1.0},
             'the': {'future': 0.25, 'past': 0.5, 'present': 0.25},
             'who': {'controls': 1.0}})
```

Considera los hints dados en la siguiente función.

In [5]:
from collections import defaultdict

def gram2prob(tokens):
    # 1) Creamos un defaultdict vacio donde guardaremos
    # cada "primera" palabra.
    ngdict = defaultdict(list)
    
    # 2) Crea tus bigrams agrupando los términos adecuadamente. Guarda 
    # El resultado en la variable 'grams'
    # Recuerda, si tokens son [x1, x2, x3, x4], nuestros grams serían
    # [(x1, x2), (x2, x3), (x3, x4)].
    #     Hint: usa la función zip. ¿Cómo agruparías la lista
    #           de arriba usando zip?
    grams = None
    
    # 3) Creamos un diccionario cuya llave es la palabra inicial
    # y el valor la palabra subsecuente. Para lograr esto, ocuparemos
    # 'ngdict', nuestro 'defaultdict'.
    # Iteramos sobre cada 'gram' en la lista 'grams'.
    # Recuerda que cada gram es un tuple con dos elementos.
    # El primer elemento de 'gram' sería la llave y el segundo elemento
    # el valor a agregar a la lista del gram dado.
    # Al final del loop, deberías tener un diccionario cuyos valores
    # es una lista de cada palabra a la llave dada.
    # Por ejemplo: para los bigrams"[(a, b), (b, c), (a, c), (b, b)]".
    # Este loop debería modificar a ngdict como:
    # {a: [b, c], b: [c, b]}
    
    # 4) iteramos cada llave en ngdict.
    # * Cuenta cuantos elementos se asocian a cada llave
    #   dentro de ngdict(longitud de cada lista). Guarda este valor
    #   dentro de la variable total_elements
    # * Accede a cada lista dentro de ngdict y usa Counter para
    #   contar el número de elementos únicos dentro de la lista.
    #   modifica esta entrada de ngdict para reemplazar la lista
    #   por el Counter
    # * Calcula la probabiliad de ocurrencia de cada elemento del contador
    #   para cada llave dada creando un diccionario:
    #   Itera  sobre cada palabra palabra del contador, accede a su elemento
    #   y divídelo entre la variable total_elements. Crea un nuevo diccionario
    #   con esto anterior y reemplaza el Contador que asignamos en el paso anterior.
    
    return ngdict

In [ ]:
# Corre esta línea para usar tu función sobre "tokens".
gram2 = gram2prob(tokens)

¿Cuáles son las probabilidades de cada posible palabra dada `dijo`?